In [1]:
##Alec's Playgound##

In [2]:
from alefuncs import *

In [38]:
def snp237(snp_number):
    """int => list
    Return the genomic position of a SNP on the GCRh37 reference genome.
    """
    
    if type(snp_number) is str \
    and snp_number.lower().startswith('rs'):
        snp_number = snp_number[2:]
    query = f"https://www.snpedia.com/index.php/Rs{snp_number}"
    html = urlopen(query).read().decode("utf-8")
    for line in html.split("\n"):
        if line.startswith('<tbody><tr><td width="90">Reference</td>'):
            reference = line.split('"')[-2]
        elif line.startswith('<tbody><tr><td width="90">Chromosome</td>'):
            chromosome = line.split("<td>")[1].split("<")[0]
        elif line.startswith('<tbody><tr><td width="90">Position</td>'):
            position = int(line.split("<td>")[1].split("<")[0])
            break

    if "GRCh38" in reference:
        lo = LiftOver("hg38", "hg19")
        return lo.convert_coordinate(f"chr{chromosome}", position)[0][:2]
    else:
        return f"chr{chromosome}", position

def get_snp_number(snp_string):
    return int(snp_string[2:])

In [39]:
#https://en.wikipedia.org/wiki/Pileup_format

longevity_snps = set([
    'rs2764264',
    'rs13217795',
    'rs2802292',
    'rs1935949',
    'rs3758391',
    'rs5882',
    'rs1042522',
    'rs1800795',
    'rs2811712',
    'rs34516635',
    'rs2542052',
    'rs3803304',
    'rs1061170',
    'rs6873545' 
])
longevity_snps = sorted([get_snp_number(snp) for snp in longevity_snps])
bam_files = {
    'ALESSIO':'/Volumes/500GB/AM_GENOME/bams/AM2.bam',
    'ALINA':'/Volumes/500GB/AM_GENOME/bams/AM1.bam',
}

for patient, bam_file in bam_files.items():
    print(f"#SNPs found on {patient}'s BAM file {bam_file.split('/')[-1]}")
    for snp in longevity_snps:
        coord = snp237(snp)
        chrom, pos = coord
        chrom = chrom[3:]
        #!samtools mpileup -r $chrom:$pos-$pos $bam_file
        r = check_output(f'samtools mpileup -r {chrom}:{pos}-{pos} {bam_file}', shell=True)
        r = r.decode().strip().split('\t')[-2].upper()
        bases = set(r)
        genotype = dict([(b,r.count(b)) for b in sorted(bases)])
        print(f'rs{snp}', coord, genotype)
        #break
    print('DONE')
    print()

#SNPs found on ALESSIO's BAM file AM2.bam
rs5882 ('chr16', 57016092) {'A': 10, 'G': 6}
rs1042522 ('chr17', 7579472) {'G': 11}
rs1061170 ('chr1', 196659237) {'C': 10, 'T': 14}
rs1800795 ('chr7', 22766645) {'C': 17, 'G': 15}
rs1935949 ('chr6', 108999287) {'G': 27}
rs2542052 ('chr11', 116699984) {'A': 7, 'C': 9}
rs2764264 ('chr6', 108934461) {'G': 1, 'T': 30}
rs2802292 ('chr6', 108908518) {'G': 18}
rs2811712 ('chr9', 21998035) {'A': 38}
rs3758391 ('chr10', 69643342) {'C': 7, 'T': 16}
rs3803304 ('chr14', 105239146) {'C': 30}
rs6873545 ('chr5', 42631264) {'T': 32}
rs13217795 ('chr6', 108974098) {'T': 27}
rs34516635 ('chr15', 99451976) {'G': 37}
DONE

#SNPs found on ALINA's BAM file AM1.bam
rs5882 ('chr16', 57016092) {'$': 1, 'A': 28, 'C': 1}
rs1042522 ('chr17', 7579472) {'G': 31}
rs1061170 ('chr1', 196659237) {'$': 1, 'C': 17, 'T': 10}
rs1800795 ('chr7', 22766645) {'G': 25}
rs1935949 ('chr6', 108999287) {'G': 25}
rs2542052 ('chr11', 116699984) {'C': 12}
rs2764264 ('chr6', 108934461) {'T': 2

In [27]:
r = check_output(f'samtools mpileup -r {chrom}:{pos}-{pos} {bam_file}', shell=True)
r = r.decode().strip().split('\t')[-2].upper()
bases = set(r)
genotype = dict([(b,r.count(b)) for b in bases])
genotype
    

{'G': 6, 'A': 10}

In [25]:
r = r.decode().strip().split('\t')[-2].upper()

'GGAAGAAGGAAAGAAA'

In [5]:
snp_number = '2764264'
query = f"https://www.snpedia.com/index.php/Rs{snp_number}"
html = urlopen(query).read().decode("utf-8")

In [16]:
bam_file = '/Volumes/500GB/AM_GENOME/bams/AM2.bam'
samtools mpileup -r 5:0-100 $bam_file

[mpileup] 1 samples in 1 input files


In [14]:
!samtools index $bam_file

In [ ]:
df = pd.DataFrame({})